# **Создание чат-бота**

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
from telegram.ext import Updater, MessageHandler, Filters
from telegram import Update
from telegram.ext import CallbackContext
from torchvision import transforms

**Параметры и устройство**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

**Архитектура модели**

In [ ]:
class Simple3DCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv3d(3, 32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d((1,2,2)),

            nn.Conv3d(32, 64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),

            nn.Flatten(),
            nn.Linear(64 * 8 * 28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        # (B, T, C, H, W) -> (B, C, T, H, W)
        x = x.permute(0, 2, 1, 3, 4)
        return self.model(x)

**Загрузка модели**

In [ ]:
model = Simple3DCNN()
model.load_state_dict(torch.load("violence_detector.pth", map_location=device))
model.to(device)
model.eval()

**Параметры обработки видео**

In [ ]:
frame_size = (112, 112)
frames_per_video = 16
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(frame_size),
    transforms.ToTensor()
])

**Функция извлечения кадров**

In [ ]:
def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = torch.linspace(0, total_frames - 1, num_frames).long()
    frames = []
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_tensor = transform(frame)
            frames.append(frame_tensor)
    cap.release()
    if len(frames) < num_frames:
        for _ in range(num_frames - len(frames)):
            frames.append(frames[-1])
    return torch.stack(frames)  # (T, C, H, W)


**Функция предсказания**

In [ ]:
def predict_violence(video_path, model, device):
    frames_tensor = extract_frames(video_path, frames_per_video)
    frames_tensor = frames_tensor.unsqueeze(0).to(device)  # (1, T, C, H, W)
    with torch.no_grad():
        outputs = model(frames_tensor)
        probs = torch.softmax(outputs, dim=1)
        pred = torch.argmax(probs, dim=1).item()
    return pred

**Обработка видео от пользователя**

In [ ]:
def handle_video(update: Update, context: CallbackContext):
    video = update.message.video or update.message.document
    if video is None:
        update.message.reply_text("Пожалуйста, отправьте видеофайл.")
        return
    
    file = video.get_file()
    file_path = f"{video.file_unique_id}.mp4"
    file.download(file_path)

    update.message.reply_text("Анализирую видео, подождите...")

    pred = predict_violence(file_path, model, device)
    os.remove(file_path)

    if pred == 1:
        update.message.reply_text("🚨 Обнаружена драка!")
    else:
        update.message.reply_text("✅ Видео безопасно.")

# **Запуск бота**

In [ ]:
def main():
    TOKEN = "7573920965:AAFVfXMiikWQmfKchv7CI20yVvFzQlBu4cQ"
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    dp.add_handler(MessageHandler(Filters.video | Filters.document.video, handle_video))

    print("Бот запущен...")
    updater.start_polling()
    updater.idle()

if __name__ == "__main__":
    main()